In [2]:
from flask import Flask, Response, render_template_string
import cv2
import numpy as np
from fer import FER
import threading
import time
import os
import sys

app = Flask("Gideon", template_folder='./templates')

# Initialize the emotion detector
emotion_detector = FER(mtcnn=True)

# Global variables
current_emotion = "No face detected"
lock = threading.Lock()
camera = None

def get_camera():
    """Get or create camera object"""
    global camera
    if camera is None:
        camera = cv2.VideoCapture(0)
        # Give camera time to initialize
        time.sleep(1)
    return camera

def detect_emotion(frame):
    """Detect emotion in a frame"""
    global current_emotion
    
    try:
        # Detect emotions
        emotions = emotion_detector.detect_emotions(frame)
        
        if emotions:
            # Get the dominant emotion
            dominant_emotion = emotions[0]['emotions']
            emotion_name = max(dominant_emotion, key=dominant_emotion.get)
            emotion_score = dominant_emotion[emotion_name]
            
            with lock:
                current_emotion = f"Detected emotion: {emotion_name.capitalize()} ({emotion_score:.2f})"
        else:
            with lock:
                current_emotion = "No face detected"
    except Exception as e:
        print(f"Error in emotion detection: {e}")
        with lock:
            current_emotion = "Error in emotion detection"

def generate_frames():
    """Generator function for video streaming"""
    global current_emotion
    
    # Try to get the camera
    try:
        cap = get_camera()
        if not cap.isOpened():
            print("Could not open webcam")
            yield (b'--frame\r\n'
                b'Content-Type: text/plain\r\n\r\n'
                b'Could not open webcam\r\n')
            return
    except Exception as e:
        print(f"Error opening camera: {e}")
        yield (b'--frame\r\n'
            b'Content-Type: text/plain\r\n\r\n'
            b'Error opening camera\r\n')
        return
    
    while True:
        try:
            success, frame = cap.read()
            if not success:
                print("Failed to capture image")
                time.sleep(0.1)
                continue
            
            # Process frame for emotion in a separate thread to avoid blocking
            emotion_thread = threading.Thread(target=detect_emotion, args=(frame.copy(),), daemon=True)
            emotion_thread.start()
            
            # Add emotion text to the frame
            with lock:
                emotion_text = current_emotion
            
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, emotion_text, (10, frame.shape[0] - 20), font, 0.8, (0, 255, 0), 2)
            
            # Encode the frame as JPEG
            ret, buffer = cv2.imencode('.jpg', frame)
            frame_bytes = buffer.tobytes()
            
            # Yield the frame in the HTTP response
            yield (b'--frame\r\n'
                b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
            
            # Wait for emotion thread to complete
            emotion_thread.join(timeout=0.1)
            
            # Small delay to reduce CPU usage
            time.sleep(0.01)
        except Exception as e:
            print(f"Error in frame generation: {e}")
            time.sleep(0.1)

@app.route('/')
def index():
    with open('templates/emotion_webcam.html', 'r') as f:
        return f.read()

@app.route('/video_feed')
def video_feed():
    """Video streaming route"""
    return Response(generate_frames(),
                    mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/emotion')
def get_emotion():
    """Return the current detected emotion"""
    with lock:
        return current_emotion

def cleanup():
    """Release resources on exit"""
    global camera
    if camera is not None and camera.isOpened():
        camera.release()
    print("Camera resources released")

if __name__ == '__main__':
    try:
        # Register cleanup handler
        import atexit
        atexit.register(cleanup)
        
        # Try different port if 5000 is in use
        port = 5000
        max_attempts = 3
        for attempt in range(max_attempts):
            try:
                print(f"Starting Flask app on port {port}")
                app.run(debug=False, threaded=True, port=port)
                break
            except OSError:
                print(f"Port {port} is in use, trying next port")
                port += 1
                if attempt == max_attempts - 1:
                    print("Could not find an available port")
                    sys.exit(1)
    except Exception as e:
        print(f"Error starting Flask app: {e}")
        cleanup()
        sys.exit(1)

Starting Flask app on port 5000
 * Serving Flask app 'Gideon'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Apr/2025 08:54:55] "GET /emotion HT